# **Taxi-v3**
Giải bài toán xe taxi tự hành theo mô tả như sau:
Môi trường là Taxi-v3 trong thư viện gym (env = gym.make("Taxi-v3").env). Vị trí taxi và vị trí hành khách sẽ được khởi tạo ngẫu nhiên. Yêu cầu là tối ưu reward nhận được (cách tính reward như trong mô tả sau: https://www.gymlibrary.dev/environments/toy_text/taxi/).


Bài nộp là một function tên là strategy nhận đầu vào là các biến theo thứ tự sau:
1. Vị trí của xe taxi (1 trong 25 vị trí trong bản đồ 5x5 của taxi-v3)
2. Vị trí của hành khách (1 trong 4 điểm R, G, B, Y trong bản đồ)
3. Vị trí trả khách (Destination: 1 trong 4 điểm R, G, B, Y trong bản đồ)
Đầu ra của function này là list các action có thể, bao gồm:
0 = south
1 = north
2 = east
3 = west
4 = pickup
5 = dropoff
Các action này cách nhau bởi dấu phẩy.

Ví dụ: strategy(3,1,R,Y) trả về 1,1,1,2,0,0,0. Trong đó, vị trí taxi là ở ô (3,1), đón khách ở ô R và trả khách ở ô Y.


## **set up environment**

In [ ]:
! pip install gym
! pip install cmake 'gym[atari]' scipy

In [ ]:
import gym
import numpy as np
import random
from tqdm import tqdm
from tqdm.notebook import trange
from IPython.display import clear_output
from time import sleep

In [ ]:
global env
env = gym.make("Taxi-v3").env

In [ ]:
# có 25 vị trí cho taxi, 5 vị trí cho khách(R,G,Y,B và trên taxi), 4 điểm dừng (R,G,Y,B)
#--> có 500 state
print("State Space {}".format(env.observation_space.n))

State Space 500


In [ ]:
# Có 6 actions:
# 0: xuống
# 1: lên
# 2: phải
# 3: trái
# 4: đón khách
# 5: trả khách
print("Action Space {}".format(env.action_space.n))

Action Space 6


**Rewards**
* -1 cho mỗi bước đi
* +20 khi đến đúng điểm trả khách
* -10 trả khách hoặc đón khách sai điểm

In [ ]:
#random state
e= env.reset()
print("State: {}".format(e))
print(env.render(mode='ansi'))


State: 43
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [ ]:
# custom  state
state = env.encode(3, 0, 2, 1)
print("State:", state)
env = env.unwrapped
env.s = state
print(env.render(mode='ansi'))

State: 309
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




## **create agent**

In [ ]:
class Agent:
    def __init__(self, env, alpha, gamma):
        self.env = env
        self.q_table = np.zeros([env.observation_space.n, env.action_space.n])
        self.alpha = alpha
        self.gamma = gamma

    def choose_action(self, state):
        return np.argmax(self.q_table[state])

    def learn(self, state, action, reward, next_state):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
        self.q_table[state, action] = new_value


## **train agent**

In [ ]:
alpha = 1
gamma = 0.9
global agent
agent = Agent(env, alpha, gamma)
best_reward = float('-inf')

In [ ]:
epsilon = 0.1

n_episodes = 100001
timesteps = 0
penalty = 0
avg_rewards = 0

for i in tqdm(range(0, n_episodes)):
    state = env.reset()
    epochs, penalties, reward, = 0, 0, 0
    done = False
    total_reward = 0
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = agent.choose_action(state)
        result = env.step(action)
        next_state, reward, done, info = result[:4]
        agent.learn(state, action, reward, next_state)

        if reward == -10:
            penalties += 1

        state = next_state
        total_reward = total_reward + reward
        epochs += 1
    if total_reward >= best_reward:
        best_reward = total_reward
    avg_rewards +=  total_reward
    timesteps += epochs
    penalty += penalties

100%|██████████| 100001/100001 [01:10<00:00, 1417.00it/s]


In [ ]:
print(f"Results after {n_episodes} episodes:")
print(f"Average timesteps per episode: {timesteps / n_episodes}")
print(f"Average penalties per episode: {penalty / n_episodes}")
print(f"Average rewards per episode: {avg_rewards / n_episodes}")
print(f"Best reward : {best_reward}")

Results after 100001 episodes:
Average timesteps per episode: 14.840251597484025
Average penalties per episode: 0.4448555514444856
Average rewards per episode: 2.156048439515605
Best reward : 15


In [ ]:
print(f"Results after {n_episodes} episodes:")
print(f"Average timesteps per episode: {timesteps / n_episodes}")
print(f"Average penalties per episode: {penalty / n_episodes}")
print(f"Average rewards per episode: {avg_rewards / n_episodes}")
print(f"Best reward : {best_reward}")

Results after 10001 episodes:
Average timesteps per episode: 14.69043095690431
Average penalties per episode: 0.4323567643235676
Average rewards per episode: 2.418358164183582
Best reward : 15


## **test**

In [ ]:
state = env.encode(3, 1, 0, 2)
env = env.unwrapped
env.s = state

In [ ]:
print(env.render(mode ='ansi'))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [ ]:
epochs = 0
penalties = 0
total_reward = 0

frames = []

done = False

while not done:

    action = agent.choose_action(state)

    result = env.step(action)
    next_state, reward, done, info = result[:4]
    agent.learn(state, action, reward, next_state)

    if reward == -10:
        penalties += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    state = next_state
    total_reward += reward
    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))
print(f"Total Reward: {total_reward}")

Timesteps taken: 10
Penalties incurred: 0
Total Reward: 11


In [ ]:

for i, frame in enumerate(frames):
  clear_output(wait = True)
  print(frame['frame'])
  print(f"Timestep: {i + 1}")
  print(f"State: {frame['state']}")
  print(f"Action: {frame['action']}")
  print(f"Reward: {frame['reward']}")
  sleep(1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 10
State: 418
Action: 5
Reward: 20


## **strategy (taxi_x,taxi_y, passenger, destination)**

In [ ]:
def strategy(taxi_x, taxi_y, passenger, destination):
  global env, agent
  if passenger == 'R': p=0
  elif passenger == 'G' : p=1
  elif passenger == 'Y' : p=2
  else : p=3

  if destination == 'R': d=0
  elif destination == 'G' : d=1
  elif destination == 'Y' : d=2
  else : d=3
  state = env.encode(taxi_x, taxi_y, p, d)
  env = env.unwrapped
  env.s = state

  epochs = 0
  penalties = 0
  total_reward = 0
  frames = []
  done = False
  while not done:
      action = agent.choose_action(state)
      result = env.step(action)
      next_state, reward, done, info = result[:4]
      agent.learn(state, action, reward, next_state)
      if reward == -10:
          penalties += 1
      frames.append({
          'frame': env.render(mode='ansi'),
          'state': state,
          'action': action,
          'reward': reward
          }
      )
      state = next_state
      total_reward += reward
      epochs += 1

  result = ','.join([str(ac['action']) for ac in frames])
  return result;

In [ ]:
print(strategy(3,1,'Y','B'))

1,3,0,0,4,1,1,2,2,2,0,0,5
